the shape of depth image and color image is different
for all data
color image shape: (720, 1280, 3)
depth image shape: (480, 848)


In [ ]:
# code to convert ros depth image to cv2 image and save it
import rospy
from cv_bridge import CvBridge, CvBridgeError
from sensor_msgs.msg import Image
import numpy as np
import cv2

def convert_depth_image(ros_image):
    cv_bridge = CvBridge()
    try:
        depth_image = cv_bridge.imgmsg_to_cv2(ros_image, desired_encoding='passthrough')
    except CvBridgeError, e:
        print e
    depth_array = np.array(depth_image, dtype=np.float32)
    np.save("depth_img.npy", depth_array)
    rospy.loginfo(depth_array)
    #To save image as png
    # Apply colormap on depth image (image must be converted to 8-bit per pixel first)
    depth_colormap = cv2.applyColorMap(cv2.convertScaleAbs(depth_image, alpha=0.03), cv2.COLORMAP_JET)
    cv2.imwrite("depth_img.png", depth_colormap)
    #Or you use 
    # depth_array = depth_array.astype(np.uint16)
    # cv2.imwrite("depth_img.png", depth_array)


def pixel2depth():
    rospy.init_node('pixel2depth',anonymous=True)
    rospy.Subscriber("/pepper/camera/depth/image_raw", Image,callback=convert_depth_image, queue_size=1)
    rospy.spin()

if __name__ == '__main__':
    pixel2depth()

In [ ]:
# try different encoding
# https://answers.ros.org/question/209000/convert-ros-image-to-opencv-image/
#cv2_img = bridge.imgmsg_to_cv2(msg, '32FC1')

In [ ]:
def Depthcallback(self,msg_depth): # TODO still too noisy!
try:
  # The depth image is a single-channel float32 image
  # the values is the distance in mm in z axis
  cv_image = self.bridge.imgmsg_to_cv2(msg_depth, "32FC1")
  # Convert the depth image to a Numpy array since most cv2 functions
  # require Numpy arrays.
  cv_image_array = np.array(cv_image, dtype = np.dtype('f8'))
  # Normalize the depth image to fall between 0 (black) and 1 (white)
  # http://docs.ros.org/electric/api/rosbag_video/html/bag__to__video_8cpp_source.html lines 95-125
  cv_image_norm = cv2.normalize(cv_image_array, cv_image_array, 0, 1, cv2.NORM_MINMAX)
  # Resize to the desired size
  cv_image_resized = cv2.resize(cv_image_norm, self.desired_shape, interpolation = cv2.INTER_CUBIC)
  self.depthimg = cv_image_resized
  cv2.imshow("Image from my node", self.depthimg)
  cv2.waitKey(1)
except CvBridgeError as e:
  print(e)